<a href="https://colab.research.google.com/github/suraj11kasasudhan/flask_api_mobilenet_classification/blob/main/flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install flask_bootstrap



     |████████████████████████████████| 460kB 5.0MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-cp37-none-any.whl size=460122 sha256=d832edaa3206f92a82f15b82d1e5ae9054669e49b37ec33658fed005b7421130
  Stored in directory: /root/.cache/pip/wheels/e8/b9/93/ef6ac3b8ead2d72cbcc042b9d58b613aaf47e533b9dc5b5999
  Created wheel for visitor: filename=visitor-0.1.3-cp37-none-any.whl size=3945 sha256=eb599ffc0d46e7118a62adaae95805f36697d167067cee739ec9e45ceb6eb684
  Stored in directory: /root/.cache/pip/wheels/68/b0/a2/cc8c7cf94ca3d1088a7d2e27936c1e0da170e05f560973e8dd
Successfully built flask-bootstrap visitor


In [ ]:
!pip install flask_wtf


     |████████████████████████████████| 174kB 6.9MB/s 


In [ ]:
from flask_bootstrap import Bootstrap
from flask_ngrok import run_with_ngrok
from flask_wtf import FlaskForm
from flask_wtf.file import FileField, FileRequired, FileAllowed
from wtforms import SubmitField
import os
from werkzeug.utils import secure_filename
from flask import Flask, render_template
from tensorflow.keras.applications.mobilenet_v2 import decode_predictions, preprocess_input, MobileNetV2
import cv2


app = Flask(__name__)
run_with_ngrok(app)   
app.config['SECRET_KEY'] = os.environ.get('SECRET_KEY') or 'you-will-never-guess'
bootstrap = Bootstrap(app)
model = MobileNetV2(input_shape=(224,224,3), weights=None)
model.load_weights('/content/static/model_weights.h5')


class UploadForm(FlaskForm):
    upload = FileField('Select an image:', validators=[
        FileRequired(),
        FileAllowed(['jpg', 'png', 'jpeg', 'JPEG', 'PNG', 'JPG'], 'Images only!')
    ])
    submit = SubmitField('Classify')


def get_prediction(img_path):
    img = cv2.imread(img_path)
    img_resized = cv2.resize(img, (224, 224))
    img_preprocessed = preprocess_input(img_resized)
    img_reshaped = img_preprocessed.reshape((1, 224, 224, 3))
    prediction = model.predict(img_reshaped)
    decoded = decode_predictions(prediction)

    top_3 = [(cat.capitalize(), round(prob*100, 2)) for (code, cat, prob) in decoded[0]][:3]
    return top_3


@app.route('/', methods=['GET', 'POST'])
def home():
    form = UploadForm()
    if form.validate_on_submit():
        f = form.upload.data
        filename = secure_filename(f.filename)
        file_url = os.path.join('static', filename
        )
        f.save(file_url)
        form = None
        prediction = get_prediction(file_url)
    else:
        file_url = None
        prediction = None
    return render_template("index.html", form=form, file_url=file_url, prediction=prediction)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b5119d295019.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Jun/2021 17:00:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 17:00:24] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Jun/2021 17:00:25] "GET / HTTP/1.1" 200 -


40960/35363 [==================================] - 0s 0us/step


127.0.0.1 - - [14/Jun/2021 17:00:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 17:00:42] "GET /static/20201119_231342_1.jpg HTTP/1.1" 200 -
